# Tarea 1 Pregunta 1

En esta tarea definiremos una funcion para romper el esquema criptográfico definido en el enunciado. Esta funcion, break_rp, que podemos ver abajo, recibe 3 valores. cipher es un texto cifrado con el esquema. freq es una lista de frecuencias de las letras del lenguaje del mensaje original. distance es una funcion que calcula la distancia entre dos caracteres.

Como no conocemos el largo de la clave, iteraremos linealmente de 1 a el tamaño del mensaje. En cada iteracion, dependiendo del tamaño de la clave n=len(k) creamos n grupos. dentro de cada unos de estos grupos agregamos las letras del mensaje que fueron codificadas con el caracter i de la llave k. Por ejemplo si tenemos una clave de tamaño 4 y un mensaje:

![](image1.png)

Grupo0: [E, _, S, _, A, E, _, I, A]

Grupo1: [S, M, A, E, _, N, C, F, D]

Grupo2: [T, E, J, S, S, D, O, I, O]

Grupo3: [E, N, E, T, I, O, D, C]

Cada uno de estos grupos deberia tener una frecuencia de letras similar al lenguaje en el que el mensaje esta escrito. Por ejemplo si la letra S tiene una frecuencia de 12.7% lo mas probable es que corresponda a la letra E en el lenguaje ingles. Podemos deducir de esto que E + K_I = S, si codifico E con la clave K entonces recibo S. Asi hemos adivinado el valor de la clave para ese grupo. Repetimos este proceso para todos los grupos y creamos una clave final.

Gracias a nuestra funcion de distancia podemos estimar si la clave que conseguimos es apropiada al comparar la frecuencia del mensaje decifrado con la frencuencia que nos entregaron. Si la distancia es pequeña entonces es probable que la llave sea la correcta y hallamos logrado decifrar el texto.

In [189]:
def break_rp(cipher: str, freq: dict[str, float], distance: callable) -> str:
    # Creamos un diccionario con todas los caracteres del lenguaje entregado
    # y reducimos las frecuencias a 0. Podemos copiar este en el futuro para
    # definir nuevos diccionarios de frecuencia vacios.
    lang = [key for key in freq.keys()]
    empty_freq_dict = freq.copy()
    for k in empty_freq_dict.keys():
        empty_freq_dict[k] = 0
    most_freq_letter = max(freq, key= lambda i: freq[i])
    print(most_freq_letter)
    # El largo de la clave es desconocido. Iteraremos desde largo 1 en adelante
    key_length = 1
    while key_length <= len(cipher):
        # Creamos una lista de frecuencias y una lista de caracteres para cada
        # uno de los caracteres de la llave. Despues compararemos la distacia
        # para encontrar el caracter en la llave correspondiente
        N = len(freq)-1
        key = ["" for i in range(key_length)]
        list_of_freqs = [empty_freq_dict.copy() for i in range(key_length)]
        character_strings = [list() for i in range(key_length)]
        for i in range(len(cipher)):
            list_of_freqs[i%key_length][cipher[i]] += 1
            character_strings[i%key_length].append(cipher[i])
        # Una vez creados los grupos, elegimos la letra mas frecuente del grupo y del lenguaje 
        # Asumimos que la diferencia entre esas letras es el valor de la clave.
        for i in range(key_length):
            most_freq_in_group = max(list_of_freqs[i], key= lambda k: list_of_freqs[i][k])
            key[i] = lang[(lang.index(most_freq_in_group) - lang.index(most_freq_letter)) % N]
            print(key)
            pass

        message = ""
        for i in range(len(cipher)):
            c = lang.index(cipher[i])
            k = lang.index(key[i % key_length])
            letter = lang[(c - k) % N]
            message += letter
        print(distance(message, freq))

    # Una vez creada una clave, deciframos el mensaje y evaluamos la frecuencia de las letras
    # en ese mensaje con la frecuencia entregada. Si es pequeña, entonces elegimos una buena clave
    # si no, seguimos iterando con una llave de mayor largo.
        key_length += 1
        if False:
            break
    return ""

La funcion de distancia que utilizaremos es la que fue definida en clases

In [190]:
def abs_distance ( string : str , frequencies : dict[str, float]) -> float :
    """
    Arguments :
    string : An abritrary string
    frequencies : A dictionary representing a character frequency
    Returns :
    distance : How distant is the string from the character frequency
    """
    return sum([
    abs ( frequencies [c] - string . count ( c ) / len ( string ) )
    for c in frequencies
    ])

Lets run an example:

In [191]:
freq = {
    "a": 1,
    "b": 7,
    "c": 1,
    "d": 2,
    "e": 1
}

print(break_rp("bcccccccdeea", freq, abs_distance))

b
['b']
-1.0
['b', '']
['b', 'b']
-1.0
['b', '', '']
['b', 'b', '']
['b', 'b', 'b']
-1.0
['a', '', '', '']
['a', 'b', '', '']
['a', 'b', 'b', '']
['a', 'b', 'b', 'b']
-1.0
['a', '', '', '', '']
['a', 'b', '', '', '']
['a', 'b', 'b', '', '']
['a', 'b', 'b', 'b', '']
['a', 'b', 'b', 'b', 'b']
-0.9999999999999982
['a', '', '', '', '', '']
['a', 'b', '', '', '', '']
['a', 'b', 'b', '', '', '']
['a', 'b', 'b', 'b', '', '']
['a', 'b', 'b', 'b', 'b', '']
['a', 'b', 'b', 'b', 'b', 'd']
-1.0
['a', '', '', '', '', '', '']
['a', 'b', '', '', '', '', '']
['a', 'b', 'b', '', '', '', '']
['a', 'b', 'b', 'b', '', '', '']
['a', 'b', 'b', 'b', 'd', '', '']
['a', 'b', 'b', 'b', 'd', 'b', '']
['a', 'b', 'b', 'b', 'd', 'b', 'b']
-1.0
['a', '', '', '', '', '', '', '']
['a', 'b', '', '', '', '', '', '']
['a', 'b', 'b', '', '', '', '', '']
['a', 'b', 'b', 'd', '', '', '', '']
['a', 'b', 'b', 'd', 'b', '', '', '']
['a', 'b', 'b', 'd', 'b', 'b', '', '']
['a', 'b', 'b', 'd', 'b', 'b', 'b', '']
['a', 'b', 'b', '